## 1. 초기 설정

In [1]:
import torch
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

torch.manual_seed(777)

learning_rate = 0.001
training_epochs = 15
batch_size = 100

## 2. training set과 Variable 만들기

In [2]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True)

## 3. model 만들기

In [7]:
class CNN(torch.nn.Module):
    def __init__(self):
        # super 메서드는 반드시 첫 번째 parameter가 현 클래스이어야 한다.
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            # torch.nn.Conv2d의 첫 번째 parameter는 색상채널을 이루는 수의 개수이고
            # 두 번째 parameter는 출력될 채널의 수를 의미한다. activation map의 개수와 같은 개념.  
            # kernel은 filter의 다른 명칭이다.
            # 나머지는 아는 거니 설명을 생략한다.
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        torch.nn.init.xavier_uniform(self.fc.weight)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        # 아래와 같이 하면 (?, 7, 7, 64) 크기의 행렬이 (?, 7 * 7 * 64) 크기의 행렬로 바뀐다.
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

model = CNN()

## 4. cost function과 optimizer 만들기

In [8]:
cost_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## 5. model 훈련시키기

In [9]:
print('Learning started. It takes sometimes.')
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = len(mnist_train)
    
    for i, (batch_xs, batch_ys) in enumerate(data_loader):
        X = Variable(batch_xs)
        Y = Variable(batch_ys)
        
        optimizer.zero_grad()
        
        Y_hat = model(X)
        cost = cost_func(Y_hat, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost.data / total_batch
        
    print("[Epoch: {:>4}] cost = {:>.9}".format(epoch + 1, avg_cost[0]))

print('Learning Finished!')

Learning started. It takes sometimes.
[Epoch:    1] cost = 0.00222329795
[Epoch:    2] cost = 0.000620203791
[Epoch:    3] cost = 0.00045753154
[Epoch:    4] cost = 0.000357311917
[Epoch:    5] cost = 0.000306011178
[Epoch:    6] cost = 0.000239420129
[Epoch:    7] cost = 0.00021286105
[Epoch:    8] cost = 0.000177525188
[Epoch:    9] cost = 0.000157953429
[Epoch:   10] cost = 0.000130264118
[Epoch:   11] cost = 0.000115634648
[Epoch:   12] cost = 9.05892157e-05
[Epoch:   13] cost = 8.13669831e-05
[Epoch:   14] cost = 8.11025966e-05
[Epoch:   15] cost = 7.25419231e-05
Learning Finished!


## 6. model 확인하기

In [10]:
model.eval()

X_test = Variable(mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float())
Y_test = Variable(mnist_test.test_labels)

prediction = model(X_test)
correct_prediction = (torch.max(prediction.data, 1)[1] == Y_test.data)
accuracy = correct_prediction.float().mean()
print('Accuracy:', accuracy)

Accuracy: 0.9834
